In [62]:
#Read file from CSV

import pandas as pd
import numpy as np
df = pd.read_csv('import_data/TestingdataFull31-08-18Type.csv')
df

,sex,Age,major group3,year,gpax,Region,previous-edu,uni-activity,influ-major,influ-people,influ-wellknown,influ-quality,influ-careermarket,TYPE
0,f,>23,Business and Law,first,B,NorthEast,highschool-mathsci,yes,mid,low,mid,mid,mid,ESFP
1,m,>23,Sciences & Maths,first,C+,North,highvocational,yes,high,mid,mid,mid,high,ENFJ
2,f,>23,Education,second,B,North,highschool-art,yes,mid,mid,high,mid,mid,ESFJ
3,m,29-34,Engineer & Industrial,completed,B+,North,highvocational,yes,high,high,high,mid,mid,ENTJ
4,m,23-28,ICTs,completed,B,North,vocational,yes,high,mid,high,high,mid,ESFJ
5,m,>23,Engineer & Industrial,second,C+,North,highschool-mathsci,no,mid,high,mid,low,mid,INTJ
6,f,>23,Sciences & Maths,second,B+,Central,highschool-mathsci,yes,high,mid,mid,mid,high,ESFJ
7,m,29-34,Engineer & Industrial,completed,B,North,highschool-mathsci,yes,high,mid,high,high,high,ESTJ
8,f,>23,Social sciences & Journal,third,B+,North,highschool-art,yes,high,low,mid,high,mid,ISFJ
9,m,>23,Business and Law,third,B+,North,vocational,yes,high,mid,mid,high,high,ISFJ


In [63]:
#function transform nominal to numeric multicoloumn

from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

# Create some toy data in a Pandas dataframe
fruit_data = pd.DataFrame({
    'fruit':  ['apple','orange','pear','orange'],
    'color':  ['red','orange','green','green'],
    'weight': [5,6,3,4]
})

class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [82]:
#Call fn transform

df = MultiColumnLabelEncoder(columns = ['sex','Age','major group3','year','gpax','Region','previous-edu','uni-activity','influ-major','influ-people','influ-wellknown','influ-quality','influ-careermarket','TYPE']).fit_transform(df)

In [66]:
# split label column

result = df['major group3']
dfwithoutlabel = df.drop(['major group3'], axis=1)

In [67]:
# split data to 8:2 training to testing

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dfwithoutlabel, result, test_size=0.2, random_state=42)

In [68]:
# train data with XGBoost

from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1,max_depth=5, random_state=99).fit(X_train, y_train)

In [69]:
# Accuracy in percentage

(clf.score(X_test, y_test))*100

86.10354223433242

In [70]:
# save model

import pickle

#serializing our model to a file called model.pkl
pickle.dump(clf, open("export_data/mbti.pkl","wb"))

#loading a model from a file called model.pkl
model = pickle.load(open("export_data/mbti.pkl","rb"))

In [81]:
# test prediction

clf.predict([[1,3,1,1,3,2,1,2,1,2,2,2,5]])

array([1])

In [84]:
df.to_csv('DataNum.csv', sep='\t', encoding='utf-8')